In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.ma as ma
from scipy.stats import linregress
import json as json
from pandas.io.json import json_normalize
import sqlite3
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify
import os

In [4]:
filepath = os.path.join("../data/cause_of_death.json")
with open(filepath) as jsonfile:
    CODjson = json.load(jsonfile)

In [5]:
CODdata = CODjson['data']
COD_df = pd.DataFrame(CODdata)
COD_df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,row-4ziw_fype-q93g,00000000-0000-0000-84F0-BE0C2CBEC77B,0,1571926784,None,1571926784,None,{ },2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,United States,169936,49.4
1,row-mf82_5qce.78c2,00000000-0000-0000-F702-31C8D4A1E5BE,0,1571926784,None,1571926784,None,{ },2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alabama,2703,53.8
2,row-7frh~9yb6-q2q4,00000000-0000-0000-6FE5-1E0821A4276B,0,1571926784,None,1571926784,None,{ },2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alaska,436,63.7
3,row-urf6.ayyb~8p5q,00000000-0000-0000-26D0-367B1AF824D1,0,1571926784,None,1571926784,None,{ },2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arizona,4184,56.2
4,row-74k2-qi6f~77p7,00000000-0000-0000-C87C-FAD80FEC2E12,0,1571926784,None,1571926784,None,{ },2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arkansas,1625,51.8


In [46]:
# json file labels
# 0 - 'sid'
# 1 - 'id'
# 2 - 'position'
# 3 - 'created_at'
# *4 - 'created_meta' - null values
# 5 - 'updated_at'
# *6 - 'meta_data' - null values
# *7 - 'meta' - only contains '{}'
# 8 - 'year'
# 9 - '113 Cause Name'
# 10 - 'Cause Name'
# 11 - 'State'
# 12 - 'Deaths'
# 13 - 'Age-Adjusted Death Rate'

sid = COD_df[0]
cod_id = COD_df[1]
position = COD_df[2]
created_at = COD_df[3]
updated_at = COD_df[5]
year = COD_df[8]
exp_cause_name = COD_df[9]
cause_name = COD_df[10]
state = COD_df[11]
deaths = COD_df[12]
Age_Adj_DR = COD_df[13]

sankey_df = pd.DataFrame({"year": year,
                            "Cause Name": cause_name,
                            "deaths": deaths,
                            "state": state
                            })
sankey_df=sankey_df[state=="United States"]
filtered_sankey_df = sankey_df[sankey_df['Cause Name'] != "All causes"]
filtered_sankey_df.head(50)
#jsonify(sankey_df)

,year,Cause Name,deaths,state
0,2017,Unintentional injuries,169936,United States
104,2017,Alzheimer's disease,121404,United States
156,2017,Stroke,146383,United States
208,2017,CLRD,160201,United States
260,2017,Diabetes,83564,United States
312,2017,Heart disease,647457,United States
364,2017,Influenza and pneumonia,55672,United States
416,2017,Suicide,47173,United States
468,2017,Cancer,599108,United States
520,2017,Kidney disease,50633,United States


In [53]:
filtered_sankey_df["deaths"] = filtered_sankey_df["deaths"].astype(int)

grouped_sankey = filtered_sankey_df.groupby(["Cause Name"]).sum()
grouped_sankey.head()

/Users/angie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,deaths
Cause Name,
Alzheimer's disease,1494816
CLRD,2594927
Cancer,10843644
Diabetes,1399943
Heart disease,12222640


In [57]:
data =[]

for index, row in grouped_sankey.iterrows():
    data.append({"from": "all deaths", "to": index, "weight": row["deaths"]})

In [58]:
print(data)

[{'from': 'all deaths', 'to': "Alzheimer's disease", 'weight': 1494816}, {'from': 'all deaths', 'to': 'CLRD', 'weight': 2594927}, {'from': 'all deaths', 'to': 'Cancer', 'weight': 10843644}, {'from': 'all deaths', 'to': 'Diabetes', 'weight': 1399943}, {'from': 'all deaths', 'to': 'Heart disease', 'weight': 12222640}, {'from': 'all deaths', 'to': 'Influenza and pneumonia', 'weight': 1094641}, {'from': 'all deaths', 'to': 'Kidney disease', 'weight': 858613}, {'from': 'all deaths', 'to': 'Stroke', 'weight': 2726523}, {'from': 'all deaths', 'to': 'Suicide', 'weight': 697016}, {'from': 'all deaths', 'to': 'Unintentional injuries', 'weight': 2347820}]
